## Synthetize multiple audio 
+ Make a command for synthesize audio file from text
+ Synthetize audio
+ Synthetize multiple audio

In [1]:
def counter(num=1, length=3):
    """Counter etc. 0001, 0002
    Attributes:
    num (int) integer etc. 1 ==> 0001
        length (int) length of counter etc. 3 ==> 001
    Return:
        (str) etc. 0001
    """
    number = '0' * length + str(num)
    number = number[len(number)-length:]
    return number

### Make a command

In [2]:
import re


def make_command(text, name, save, quality):    
    return re.sub(" +", " ", f"""
    source .venv/bin/activate && \
    python main.py \
        -t "{text}" \
        -n "{name}" \
        -q "{quality}" \
        -s "{save}"
    """).strip()

text = "Hello World"
name = "kathleen-glow_tts"
save = "/root/Desktop/sample.wav"
quality = "low"

cmd = make_command(text, name, save, quality)
cmd

'source .venv/bin/activate && python main.py -t "Hello World" -n "kathleen-glow_tts" -q "low" -s "/root/Desktop/sample.wav"'

### Synthetize audio

In [3]:
import subprocess as sb

def synthesize_audio(cmd, cwd):
    process = sb.Popen(
        cmd, cwd=cwd, shell=True, 
        executable='/bin/bash', encoding='UTF-8',
        stdin=sb.PIPE, stdout=sb.PIPE, stderr=sb.PIPE
    )
    stdout, stderr = process.communicate('')

cwd = "/root/PycharmProjects/Pytorch/03 - Larynx TTS"

# synthesize_audio(cmd, cwd)

### Synthetize multiple audio

In [4]:
def load_text(meta):
    with open(meta, "r", encoding="utf-8") as file:
        texts = [line.split('|')[-1] for line in file.read().splitlines()]
    return texts

meta = "/root/PycharmProjects/Tensorflow-2.X/07 - DeepSpeech Vad Transcriber + Create Wavs/text.txt"

load_text(meta)

['Good morning! How are you?',
 'I’m fine, thanks. How are you?',
 'Hello! How are you doing?',
 'Fine, just fine. How are you?',
 'Hi, how is it going?',
 'Pretty good. How about you?',
 'Hi, what’s up?',
 'Not much. What about you?',
 'What have you been up to?',
 'The same as usual.',
 'How to thank people?',
 'Thank you for looking after my son.',
 'You’re very welcome.',
 'Thanks for your help.',
 'No problem.',
 'Thank you for carrying my bag.',
 'Don’t mention it.',
 'That’s very kind of you, but you really shouldn’t have gone to all this trouble.',
 'I was happy to help.']

### Transcribe audio

In [5]:
import deepspeech
import pydub

class DeepSpeech:
    def __init__(self, model, scorer, alpha=0.931289039105002, beta=1.1834137581510284):
        self.model = deepspeech.Model(model)
        self.model.enableExternalScorer(scorer)
        self.model.setScorerAlphaBeta(alpha, beta)
   
    def transcribe(self, audio):
        audio_segment = pydub.AudioSegment.from_wav(audio)
        audio_segment = audio_segment.set_frame_rate(16000)
        assert audio_segment.frame_rate == 16000 
        samples = audio_segment.get_array_of_samples()
        return self.model.stt(samples)

    
#model_path = 'model/output_graph.tflite'
#scorer_path = 'model/output_graph.scorer'
#audio_path = "model/sample.wav"

#model = DeepSpeech(model_path, scorer_path)
#model.transcribe(audio_path)

In [6]:
import os
import re

from nltk.translate.bleu_score import sentence_bleu

# FUNCTIONS
def compute_bleu(candidate, reference, gram=1):
    candidate = candidate.split(" ")
    reference = [reference.split(" ")]
    weights = [0, 0, 0, 0]
    weights[gram - 1] = 1 
    return sentence_bleu(reference, candidate, weights=weights) 


def make_command(text, name, save, quality):    
    return re.sub(" +", " ", f"""
    source .venv/bin/activate && \
    python main.py \
        -t "{text}" \
        -n "{name}" \
        -q "{quality}" \
        -s "{save}"
    """).strip()

def filter_text(text):
    # Filter chars [a-z' ]
    text = text.lower()
    text = re.sub(r"’m", r"'m", text)
    text = re.sub(r"’s", r"'s", text)
    text = re.sub(r"’re", r"'re", text)
    text = re.sub(r"’ve", r"'ve", text)
    text = re.sub(r"[^\w ']", "", text)     # a-z'
    text = re.sub(" +", " ", text)          # "  " -> " "
    return text.strip()


# PATHS
model_path = 'model/output_graph.tflite'
scorer_path = 'model/output_graph.scorer'
base = os.path.abspath(os.path.dirname(__name__))
names = f"/root/PycharmProjects/Pytorch/03 - Larynx TTS/voices/en-us"
text_path = "/root/PycharmProjects/Tensorflow-2.X/07 - DeepSpeech Vad Transcriber + Create Wavs/text.txt"
cwd = "/root/PycharmProjects/Pytorch/03 - Larynx TTS"
save = "/root/Desktop/wavs"
quality = "high"
meta = "/root/Desktop/metadata.csv"

# DIRECTORIES
os.makedirs(save, exist_ok=True)

# VARIABLES
count = 1
model = DeepSpeech(model_path, scorer_path)

with open(meta, "w", encoding="utf-8") as file:    
    for text in load_text(text_path):
        for speaker in os.listdir(names):
            name = f"SA-001-{counter(count, 4)}"
            audio_path = f"{save}/{name}.wav"
            cmd = make_command(text, speaker, audio_path, quality)
            synthesize_audio(cmd, cwd)            
            transcript = model.transcribe(audio_path)
            
            text = filter_text(text)
            bleu = compute_bleu(transcript, text, gram=1)
            print(transcript, text, bleu)
            print()
            file.writelines(f"{name}|{text}|{transcript}|{bleu}\n")
            count += 1
            
load_text(meta)

TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85


good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

morning how are you good morning how are you 0.7788007830714049

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are  good morning how are you 0.8



/root/anaconda3/envs/DeepSpeech/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


suborning how are you good morning how are you 0.5841005873035536

morning how are you good morning how are you 0.7788007830714049

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0



/root/anaconda3/envs/DeepSpeech/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/anaconda3/envs/DeepSpeech/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


but moment how i good morning how are you 0.19470019576785122

good morning how are you good morning how are you 1.0

good morning how are good morning how are you 0.7788007830714049

good morning how are you good morning how are you 1.0

the morning how are you good morning how are you 0.8

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

could morning how are you good morning how are you 0.8

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

good morning how are you good morning how are you 1.0

fine thanks how are you i'm fine thanks how are you 0.8187307530779819

i'm fine thanks how are you i'm fine thanks how are you 1.0

in fine thanks how are ye i'm fine thanks how are you 0.6666666666666666

i'm fine thanks how are you i'm fine thanks how are you 1.0

pretty good about you pretty good how about you 0.7788007830714049

for gigaboo pretty good how about you 0

here cato pretty good how about you 0

pretty did how about you pretty good how about you 0.8

what hi what's up 0

i was hi what's up 0

how hi what's up 0

i was up hi what's up 0.3333333333333333

howard hi what's up 0

what hi what's up 0

i was  hi what's up 0

high water hi what's up 0

high boots are hi what's up 0

i want  hi what's up 0

high was hi what's up 0

i was hi what's up 0

i was hi what's up 0

his  hi what's up 0

i was hi what's up 0

i was hi what's up 0

had hi what's up 0

i was hi what's up 0

it hi what's up 0

i was hi what's up 0

cibot hi what's up 0

i was hi what's up 0

i was hi what's up 0

i was hi what's up 0

high but hi what's up 0

i was hi what's up 0

i went up hi what's up 0.3333333333333333

not much what about you not much what about you 1.0

no but whitby not much what about you 0

not much what about you not much what about you 1.0



thank you for looking after my son thank you for looking after my son 1.0

you're very welcome you're very welcome 1.0

a very welcome you're very welcome 0.6666666666666666

you are very welcome you're very welcome 0.5

you are very welcome you're very welcome 0.5

you're very welcome you're very welcome 1.0

he very well come you're very welcome 0.25

very well you're very welcome 0.3032653298563167

you are very well to you're very welcome 0.2

very welcome you're very welcome 0.6065306597126334

your very welcome you're very welcome 0.6666666666666666

you're very welcome you're very welcome 1.0

every vice you're very welcome 0

you are very welcome you're very welcome 0.5

we welcome you're very welcome 0.3032653298563167

very well come you're very welcome 0.3333333333333333

you are very welcome you're very welcome 0.5

you are very welcome you're very welcome 0.5

you're very welcome you're very welcome 1.0

o very well you're very welcome 0.3333333333333333

you are very welc

that's very kind of you but really chantant all this trouble that's very kind of you but you really shouldnt have gone to all this trouble 0.6319490258171626

that's very kind of you but you really shown have gone to all this trouble that's very kind of you but you really shouldnt have gone to all this trouble 0.9333333333333333

that's very kind of you but you really shall a gone to all this trouble that's very kind of you but you really shouldnt have gone to all this trouble 0.8666666666666667

that's very kind of you but you really content all this trouble that's very kind of you but you really shouldnt have gone to all this trouble 0.7139007178154544

that's very kind of you but you really shall have gone to all this trouble that's very kind of you but you really shouldnt have gone to all this trouble 0.9333333333333333

that's very kind of you but you really shall go to all this trouble that's very kind of you but you really shouldnt have gone to all this trouble 0.798053811174876

['1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.7788007830714049',
 '1.0',
 '1.0',
 '0.8',
 '0.5841005873035536',
 '0.7788007830714049',
 '1.0',
 '1.0',
 '1.0',
 '0.19470019576785122',
 '1.0',
 '0.7788007830714049',
 '1.0',
 '0.8',
 '1.0',
 '1.0',
 '1.0',
 '0.8',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.8187307530779819',
 '1.0',
 '0.6666666666666666',
 '1.0',
 '1.0',
 '0.15163266492815836',
 '1.0',
 '1.0',
 '0.6549846024623855',
 '0.5',
 '0.8333333333333334',
 '1.0',
 '0.7142857142857143',
 '0.5',
 '0.3274923012311928',
 '0',
 '1.0',
 '0.8333333333333334',
 '0.49123845184678916',
 '0.6666666666666666',
 '0',
 '0.8333333333333334',
 '0.8333333333333334',
 '0.8187307530779819',
 '0.8333333333333334',
 '0.8333333333333334',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.7788007830714049',
 '1.0',
 '0.8',
 '0.7788007830714049',
 '1.0',
 '1.0',
 '0.7788007830714049',
 '1.0',
 '0.7788007830714049',
 '1.0',
 '0.7788007830714049',
 '0.8',
 '1.0',
 '1.0',
 '1.0',
 '0.8',
 '0.8',
 '1.0',
 '0.8',
 '0.34227807935506